In [126]:
import pandas as pd
import numpy as np

In [127]:
homicidios = pd.read_csv("data/censo_2000_pe/agressoes_arma_de_fogo_2000.csv")
homicidios = homicidios.rename(index=str, columns={"2000": "ano_2000"})
homicidios = homicidios.replace('-', np.nan)
homicidios['ano_2000'] = homicidios.ano_2000.astype("float64")
#homicidios

In [128]:
populacao = pd.read_csv("data/estimativa_populacao/populacao_pe_municipios.csv")
#populacao

In [129]:
microrregioes = pd.read_csv("data/microrregioes/municipios_microrregioes_pernambuco.csv")
#microrregioes

In [130]:
aux2 = pd.read_csv("data/censo_2000_pe/csv/edu_alunos_por_serie.csv")
#aux2.head()
codigo = aux2[['municipio', 'codigo_unidade_geografica']]
codigo = codigo.copy()
codigo['codigo_unidade_geografica'] = codigo['codigo_unidade_geografica'].apply(lambda x: int(x/10))
#codigo

In [131]:
codigo['municipio'] = codigo['municipio'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')
microrregioes['municipio'] = microrregioes['municipio'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')
microrregioes['microrregiao'] = microrregioes['microrregiao'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')
populacao['municipio'] = populacao['municipio'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')


In [138]:
tb_parcial = pd.concat([codigo.set_index("municipio"),\
                                microrregioes.set_index("municipio"), populacao.set_index("municipio")], axis= 1)
tb_parcial = tb_parcial.reset_index()
tb_parcial = tb_parcial.rename(index=str, columns={"index": "municipio"})

In [147]:
tb_parcial.head()


,municipio,codigo_unidade_geografica,microrregiao,pop
0,Abreu e Lima,260005,Recife,83486
1,Afogados da Ingazeira,260010,Vale do Pajeu,34524
2,Afranio,260020,Petrolina,14227
3,Agrestina,260030,Brejo,18713
4,Agua Preta,260040,Mata Meridional,25353


In [148]:
def filtro(codigo):
    aux=[]
    if (codigo >= 260001 and codigo <= 269999):
        aux.append(int(codigo))
    df_newcode = pd.DataFrame(aux, columns=["codigo_un"])
    return df_newcode

In [149]:
n_homicidios = pd.concat(map(lambda a: filtro(a.codigo_unidade_geografica).\
                             assign(homicidio = a.ano_2000), homicidios.itertuples()), ignore_index=True)
#n_homicidios.head()

In [150]:
tabela_ref = pd.concat([tb_parcial.set_index("codigo_unidade_geografica"),\
                                n_homicidios.set_index("codigo_un")], axis= 1)

In [151]:
tabela_ref.head()

,municipio,microrregiao,pop,homicidio
260005,Abreu e Lima,Recife,83486,32.0
260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0
260020,Afranio,Petrolina,14227,1.0
260030,Agrestina,Brejo,18713,8.0
260040,Agua Preta,Mata Meridional,25353,21.0


In [152]:
tabela_ref = tabela_ref.reset_index()
tabela_ref = tabela_ref.rename(columns={"index": "codigo_un"})
tabela_ref.head()

,codigo_un,municipio,microrregiao,pop,homicidio
0,260005,Abreu e Lima,Recife,83486,32.0
1,260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0
2,260020,Afranio,Petrolina,14227,1.0
3,260030,Agrestina,Brejo,18713,8.0
4,260040,Agua Preta,Mata Meridional,25353,21.0


In [153]:
def taxa(populacao, homicidios):
    aux=[]
    taxa_h = 100000*homicidios/populacao
    aux.append(taxa_h)
    df_taxa = pd.DataFrame(aux, columns=["taxa_homicidio"])
    return df_taxa

In [154]:
taxa = pd.concat(map(lambda a: taxa(a.pop, a.homicidio),\
                       tabela_ref.itertuples()), ignore_index=True)

In [155]:
tabela = pd.concat([tabela_ref, taxa], axis=1)
tabela.head()

,codigo_un,municipio,microrregiao,pop,homicidio,taxa_homicidio
0,260005,Abreu e Lima,Recife,83486,32.0,38.329780
1,260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0,23.172286
2,260020,Afranio,Petrolina,14227,1.0,7.028889
3,260030,Agrestina,Brejo,18713,8.0,42.751029
4,260040,Agua Preta,Mata Meridional,25353,21.0,82.830434


In [229]:
#tabela.to_csv('../2017-2-projeto-lbs4-eebls-ktcn/tabela_final.csv', index=False)
#tabela = pd.read_csv("../2017-2-projeto-lbs4-eebls-ktcn/tabela_final.csv")

In [230]:
def retirar_acento(dataframe):
    dataframe['municipio'] = dataframe['municipio'].str.normalize('NFKD').\
                                                        str.encode('ascii', errors='ignore').str.decode('utf-8')
    dataframe = dataframe.set_index('municipio')
    return dataframe

In [231]:
data_stack=[]

In [232]:
data = pd.read_csv("data/censo_2000_pe/csv/edu_alfabetizacao.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/edu_alunos_por_serie.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/edu_pessoas_por_ano_de_estudo.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_60_anos_contribuintes_no_rendimento_mensal.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_aposentados_ou_pensionistas.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_criancas_14_anos_com_responsaveis_menos_4_anos_estudo.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_destino_do_lixo.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_domicilios_particulares_por_rendimento_medio_mensal.csv")
data_stack.append(data)
#data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_domicilios_rurais_com_agua.csv")
#data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_domicilios_urbanos_com_agua.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_esgotamento_sanitario.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_moradores_por_domicilio.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_pessoas_com_menos_de_4_anos_de_estudo.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_pessoas_ocupadas.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_pessoas_ocupadas_contribuintes.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_pessoas_ocupadas_por_ocupacao.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_populacao.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_populacao_por_cor.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_populacao_residente.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/indic_soc_taxa_de_escolarizacao.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/trab_empregados_trabalho_principal.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/trab_horas_trabalhadas.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/trab_pessoas_economicamente_ativas_e_ocupadas.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/csv/trab_rendimento_mediano_mensal.csv")
data_stack.append(data)

In [233]:
for x in range(0,23):
    data_stack[x] = retirar_acento(data_stack[x])

In [234]:
data_s = pd.concat(data_stack, axis=1)
#teste.head()

In [235]:
tabela = tabela.set_index('municipio')
#tabela.head()

In [240]:
tabela_final = pd.concat([tabela,data_s], axis=1).drop('codigo_unidade_geografica', 1)
tabela_final.head()

,codigo_un,microrregiao,pop,homicidio,taxa_homicidio,alfabetizados,taxa_alfabetizacao,pop_10_a_14_anos,pop_10_a_14_anos_alfabetizadas,pop_10_a_14_anos_taxa_alfabetizacao,...,pessoas_10_anos_ou_mais_ocupadas_total,pessoas_10_anos_ou_mais_ocupadas_homens,pessoas_10_anos_ou_mais_ocupadas_mulheres,valor_do_rendimento_nominal_mediano_mensal_de_pessoas_de_10_anos_ou_mais_total,valor_do_rendimento_nominal_mediano_mensal_de_pessoas_de_10_anos_ou_mais_empregadas_total,valor_do_rendimento_nominal_mediano_mensal_de_pessoas_de_10_anos_ou_mais_empregadas_com_carteira_assinada,valor_do_rendimento_nominal_mediano_mensal_de_pessoas_de_10_anos_ou_mais_empregadas_militares_e_funcionarios_publicos_estatuarios,valor_do_rendimento_nominal_mediano_mensal_de_pessoas_de_10_anos_ou_mais_empregadas_outros_sem_carteira_assinada,valor_do_rendimento_nominal_mediano_mensal_de_pessoas_de_10_anos_ou_mais_empregadores,valor_do_rendimento_nominal_mediano_mensal_de_pessoas_de_10_anos_ou_mais_conta_propria
Abreu e Lima,260005,Recife,83486,32.0,38.329780,63 709,88,0,8 979,8 513,...,26 071,16 766,9 306,224,0,226,0,260,0,500
Afogados da Ingazeira,260010,Vale do Pajeu,34524,8.0,23.172286,19 629,74,9,3 807,3 424,...,12 373,7 747,4 626,151,0,151,0,151,0,236
Afranio,260020,Petrolina,14227,1.0,7.028889,8 158,68,9,1 824,1 683,...,4 726,3 295,1 430,150,0,150,0,186,0,200
Agrestina,260030,Brejo,18713,8.0,42.751029,9 599,61,0,2 263,1 811,...,7 576,4 532,3 044,150,0,150,0,180,0,170
Agua Preta,260040,Mata Meridional,25353,21.0,82.830434,14 347,65,7,3 784,3 114,...,5 043,3 487,1 557,151,0,151,0,151,0,160


In [241]:
list(tabela_final)

['codigo_un',
 'microrregiao',
 'pop',
 'homicidio',
 'taxa_homicidio',
 'alfabetizados',
 'taxa_alfabetizacao',
 'pop_10_a_14_anos',
 'pop_10_a_14_anos_alfabetizadas',
 'pop_10_a_14_anos_taxa_alfabetizacao',
 'pop_10_a_19_anos',
 'pop_10_a_19_anos_alfabetizadas',
 'pop_10_a_19_anos_taxa_alfabetizacao',
 'pop_20_anos_ou_mais',
 'pop_20_anos_ou_mais_alfabetizadas',
 'pop_20_anos_ou_mais_taxa_alfabetizacao',
 'alunos_total',
 'alunos_creche',
 'aluno_pre_escolar',
 'alunos_alfabetizacao_de_adultos',
 'alunos_fundamental',
 'alunos_medio',
 'alunos_pre_vestibular',
 'alunos_graduacao',
 'alunos_mestrado_ou_doutorado',
 'pessoas_por_anos_de_estudo_sem_instrucao_e_menos_de_1_ano',
 'pessoas_por_anos_de_estudo_1_a_3_anos',
 'pessoas_por_anos_de_estudo_4_anos',
 'pessoas_por_anos_de_estudo_5_a_7_anos',
 'pessoas_por_anos_de_estudo_8_anos',
 'pessoas_por_anos_de_estudo_9_e_10_anos',
 'pessoas_por_anos_de_estudo_11_anos',
 'pessoas_por_anos_de_estudo_12_anos',
 'pessoas_por_anos_de_estudo_13_an

In [242]:
tabela_final.to_csv('../2017-2-projeto-lbs4-eebls-ktcn/tabela_final.csv', index=False)